# 🚗 Racecar Gym (3D, PyBullet) — Google Colab Quickstart

This notebook installs **racecar_gym** (PyBullet + Gymnasium), downloads tracks, and runs a **headless** rollout that saves an MP4. It also shows how to enable the **RGB camera** via a custom scenario and how to launch **multi-agent** (PettingZoo) later.

> Tip: In Colab, set **Runtime → Change runtime type → GPU (optional)**. Everything works on CPU too.

## 1) Install Python dependencies
We pin versions to match the library's `requirements.txt` so it stays stable in Colab.

In [1]:
!pip -q install -U pip setuptools wheel
# Versions aligned with the repo's requirements.txt
!pip -q install pybullet==3.2.5 gymnasium==0.28.1 numpy==1.22.3 scipy yamldataclassconfig==1.5.0 "nptyping<2.0" pettingzoo==1.22.3 imageio[ffmpeg] matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [6]:
import numpy as np
print(np.__version__)

2.0.2


In [5]:
!

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pybullet'


## 2) Clone & install `racecar_gym`
This uses editable mode so you can tweak scenarios/configs if you want.

In [ ]:
!git clone https://github.com/axelbr/racecar_gym.git
%cd racecar_gym
!pip -q install -e .
%cd /content

## 3) (Optional) Manually pre-download all tracks
Tracks normally auto-download on first use, but you can prefetch them to avoid delays during `env.reset()`.

In [ ]:
%%bash
set -e
cd racecar_gym/models/scenes
VERSION=v1.0.0
if [ ! -d ./austria ]; then
  wget -q https://github.com/axelbr/racecar_gym/releases/download/tracks-${VERSION}/all.zip
  unzip -q -o all.zip
fi
echo "Tracks ready."

## 4) Smoke test (single-agent, headless)
Creates the built-in **`SingleAgentAustria-v0`** env with a following camera (`rgb_array_follow`) and runs a short random rollout, saving a video.

In [ ]:
import gymnasium as gym
import racecar_gym.envs.gym_api  # registers envs
import numpy as np
import imageio.v2 as imageio
from pathlib import Path

env = gym.make(
    id="SingleAgentAustria-v0",
    render_mode="rgb_array_follow",
    render_options=dict(width=640, height=360, agent="A")
)

obs, info = env.reset(options=dict(mode="grid"))
print("Observation keys:", list(obs.keys()))

frames = []
for t in range(300):
    # Action space is a Dict; default has keys 'motor' and 'steering', both in [-1, 1]
    action = {
        "A": {
            "motor": np.array([0.6], dtype=np.float32),     # moderate throttle
            "steering": np.array([0.0], dtype=np.float32),  # straight
        }
    }
    obs, reward, terminated, truncated, state = env.step(action)
    # Ask the env for a frame (works when render_mode is 'rgb_array_*')
    frame = env.render()
    if frame is not None:
        frames.append(frame)
    if bool(terminated) or bool(truncated):
        break

env.close()

out = Path("/content/racecar_single_agent_demo.mp4")
if frames:
    imageio.mimwrite(out, frames, fps=30, quality=8)
    print("Saved video to:", out)
else:
    print("No frames captured. Try switching to smaller width/height in render_options.")

## 5) Custom scenario with RGB camera in observations
The default scenario may not include the RGB camera in `obs`. Here we write a **YAML** scenario that enables the `rgb_camera` sensor, then load it with the generic `SingleAgentRaceEnv-v0` id.

In [ ]:
from pathlib import Path
scenario_dir = Path("/content/my_scenarios")
scenario_dir.mkdir(parents=True, exist_ok=True)

scenario_text = """
world:
  name: austria
agents:
  - id: A
    vehicle:
      name: racecar
      sensors: [lidar, pose, velocity, acceleration, rgb_camera]
      actuators: [motor, steering]
      color: blue
    task:
      task_name: maximize_progress
      params: {laps: 1, time_limit: 30.0, terminate_on_collision: False}
"""

scenario_path = scenario_dir / "austria_rgb.yml"
scenario_path.write_text(scenario_text)
print("Wrote", scenario_path)

import gymnasium as gym
import racecar_gym.envs.gym_api

env = gym.make(
    id="SingleAgentRaceEnv-v0",
    scenario=str(scenario_path),
    render_mode="rgb_array_follow",
    render_options=dict(width=640, height=360, agent="A")
)

obs, info = env.reset(options=dict(mode="grid"))
print("Observation keys:", list(obs["A"].keys()) if "A" in obs else list(obs.keys()))

# If camera is present, display one frame
frame = env.render()
import matplotlib.pyplot as plt
if frame is not None:
    plt.figure(figsize=(8,4.5))
    plt.imshow(frame)
    plt.axis("off")
    plt.show()

env.close()

## 6) Multi-agent preview (PettingZoo API)
You can switch to the PettingZoo API for multi-agent training later.

In [ ]:
from racecar_gym.envs import pettingzoo_api
from pathlib import Path

# Reuse our custom camera scenario for multi-agent by duplicating the agent
multi_yaml = """
world:
  name: austria
agents:
  - id: A
    vehicle:
      name: racecar
      sensors: [lidar, pose, velocity, acceleration, rgb_camera]
      actuators: [motor, steering]
      color: blue
    task:
      task_name: maximize_progress
      params: {laps: 1, time_limit: 20.0, terminate_on_collision: False}
  - id: B
    vehicle:
      name: racecar
      sensors: [lidar, pose, velocity, acceleration, rgb_camera]
      actuators: [motor, steering]
      color: red
    task:
      task_name: maximize_progress
      params: {laps: 1, time_limit: 20.0, terminate_on_collision: False}
"""

mpath = Path("/content/my_scenarios/austria_rgb_multi.yml")
mpath.write_text(multi_yaml)

env = pettingzoo_api.env(scenario=str(mpath))
env.reset()

for agent in env.agent_iter(max_iter=200):
    obs, rew, term, trunc, info = env.last()
    # Random actions shaped like the env expects
    if term or trunc:
        action = None
    else:
        # The library uses normalized [-1,1] motor/steering values internally
        action = {"motor": 0.5, "steering": 0.0}
    env.step(action)

env.close()
print("PettingZoo multi-agent loop finished.")

## 7) Notes & gotchas
* If Colab warns about a **NumPy** mismatch after installation, go to **Runtime → Restart runtime**, then rerun.
* Headless rendering: use `render_mode='rgb_array_follow'` or inspect `obs['A']['rgb_camera']` if camera is enabled.
* Actions are normalized in **[-1, 1]** for both `motor` and `steering`.
* Tracks auto-download on first use. You can prefetch with the cell above if desired.